# 第四章 基于LangChain的文档问答
本章内容主要利用langchain构建向量数据库，可以在文档上方或关于文档回答问题，因此，给定从PDF文件、网页或某些公司的内部文档收集中提取的文本，使用llm回答有关这些文档内容的问题

## 环境配置

安装langchain，设置chatGPT的OPENAI_API_KEY
* 安装langchain
```
pip install langchain
```
* 安装docarray
```
pip install docarray
```
* 设置API-KEY环境变量
```
export OPENAI_API_KEY='api-key'

```

In [1]:
import os
import sys
sys.path.append("D:\\software\\conda\\envs\\torch2\\Lib\\site-packages")

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) #读取环境变量

In [11]:
!pip install python-dotenv docarray

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/220.8 kB ? eta -:--:--
     - -------------------------------------- 10.2/220.8 kB ? eta -:--:--
     - -------------------------------------- 10.2/220.8 kB ? eta -:--:--
     - -------------------------------------- 10.2/220.8 kB ? eta -:--:--
     - -------------------------------------- 10.2/220.8 kB ? eta -:--:--
     - -------------------------------------- 10.2/220.8 kB ? eta -:--:--
     - -------------------------------------- 10.2/220.8 kB ? eta -:--:--
     - -------------------------------------- 10.2/220.8 kB ? eta -:--:--
     - -------------------------------------- 10.2/220.8 kB ? eta -:--:--
     ------------ ------------------------ 71.7/220.8 kB 151.3 kB/s eta 0:00:01
     ------------------ ----------------- 112.6/220.8 kB 218.8 kB/s eta 0:00:01
     ------------------ ----------------- 112.6/220.8 kB 218.8 kB/s eta 0:00:01
     ------------------ ----------

In [4]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003",max_tokens=1024)
llm("怎么评价人工智能")

'\n\n人工智能是一种令人印象深刻的技术，它不仅能够利用机器学习和数据挖掘技术提供更先进的解决方案，而且可以模仿人类大脑的思考模式，从而让计算机变得更加智能。它的出现已经极大地提高了我们的生活质量，改善了我们的工作效率，并且给我们带来了许多新的机会。'

### 导入embedding模型和向量存储组件
使用Dock Array内存搜索向量存储，作为一个内存向量存储，不需要连接外部数据库

In [2]:
from langchain.chains import RetrievalQA #检索QA链，在文档上进行检索
from langchain.chat_models import ChatOpenAI #openai模型
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain.vectorstores import DocArrayInMemorySearch #向量存储
from IPython.display import display, Markdown #在jupyter显示信息的工具

In [3]:
#读取文件
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [4]:
#查看数据
import pandas as pd
data = pd.read_csv(file,header=None)
data

,0,1,2
0,NaN,name,description
1,0.0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
2,1.0,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
3,2.0,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
4,3.0,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
...,...,...,...
996,995.0,"Men's Classic Denim, Standard Fit",Crafted from premium denim that will last wash...
997,996.0,CozyPrint Sweater Fleece Pullover,The ultimate sweater fleece - made from superi...
998,997.0,Women's NRS Endurance Spray Paddling Pants,These comfortable and affordable splash paddli...
999,998.0,Women's Stop Flies Hoodie,This great-looking hoodie uses No Fly Zone Tec...


In [5]:
data.head(10)

,0,1,2
0,NaN,name,description
1,0.0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
2,1.0,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
3,2.0,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
4,3.0,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
5,4.0,EcoFlex 3L Storm Pants,Our new TEK O2 technology makes our four-seaso...
6,5.0,"Smooth Comfort Check Shirt, Slightly Fitted",Our men's slightly fitted check shirt is the p...
7,6.0,Mens Outdoor Research Apex Gloves,These Outdoor Research Arete gloves are perfec...
8,7.0,"Women's Ultraplush Sweats, Relaxed Crewneck",This crewneck sweatshirt is crafted from soft ...
9,8.0,Mountain Man Fleece Jacket,Our best-value fleece jacket is designed with ...


提供了一个户外服装的CSV文件，我们将使用它与语言模型结合使用

#### 创建向量存储
将导入一个索引，即向量存储索引创建器

In [6]:
from langchain.indexes import VectorstoreIndexCreator #导入向量存储索引创建器

In [7]:
'''
将指定向量存储类,创建完成后，我们将从加载器中调用,通过文档记载器列表加载
'''

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [8]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [9]:
response = index.query(query)#使用索引查询创建一个响应，并传入这个查询

In [10]:
display(Markdown(response))#查看查询返回的内容



| Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | UPF 50+ rated, 100% polyester, wrinkle-resistant, front and back cape venting, two front bellows pockets |
| Men's Plaid Tropic Shirt, Short-Sleeve | UPF 50+ rated, 52% polyester and 48% nylon, machine washable and dryable, front and back cape venting, two front bellows pockets |
| Men's TropicVibe Shirt, Short-Sleeve | UPF 50+ rated, 71% Nylon, 29% Polyester, 100% Polyester knit mesh, machine wash and dry, front and back cape venting, two front bellows pockets |
| Sun Shield Shirt by | UPF 50+ rated, 78% nylon, 22% Lycra Xtra Life fiber, handwash, line dry, wicks moisture, fits comfortably over swimsuit, abrasion resistant |

All four shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant

得到了一个Markdown表格，其中包含所有带有防晒衣的衬衫的名称和描述，还得到了一个语言模型提供的不错的小总结

#### 使用语言模型与文档结合使用
想要使用语言模型并将其与我们的许多文档结合使用，但是语言模型一次只能检查几千个单词，如果我们有非常大的文档，如何让语言模型回答关于其中所有内容的问题呢？通过embedding和向量存储实现
* embedding   
文本片段创建数值表示文本语义，相似内容的文本片段将具有相似的向量，这使我们可以在向量空间中比较文本片段
* 向量数据库    
向量数据库是存储我们在上一步中创建的这些向量表示的一种方式，我们创建这个向量数据库的方式是用来自传入文档的文本块填充它。
当我们获得一个大的传入文档时，我们首先将其分成较小的块，因为我们可能无法将整个文档传递给语言模型，因此采用分块embedding的方式储存到向量数据库中。这就是创建索引的过程。

注意： 这里的向量数据库大家可以了解一下腾松说的—zilliz-cloud

通过运行时使用索引来查找与传入查询最相关的文本片段，然后我们将其与向量数据库中的所有向量进行比较，并选择最相似的n个，返回语言模型得到最终答案

In [11]:
#创建一个文档加载器，通过csv格式加载
loader = CSVLoader(file_path=file)
docs = loader.load()

In [12]:
docs[0]#查看单个文档，我们可以看到每个文档对应于CSV中的一个块

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [13]:
'''
因为这些文档已经非常小了，所以我们实际上不需要在这里进行任何分块,可以直接进行embedding
'''

from langchain.embeddings import OpenAIEmbeddings #要创建可以直接进行embedding，我们将使用OpenAI的可以直接进行embedding类
embeddings = OpenAIEmbeddings() #初始化

In [14]:
embed = embeddings.embed_query("Hi my name is Harrison")#让我们使用embedding上的查询方法为特定文本创建embedding

In [15]:
print(len(embed))#查看这个embedding，我们可以看到有超过一千个不同的元素

1536


In [16]:
print(embed[:5])#每个元素都是不同的数字值，组合起来，这就创建了这段文本的总体数值表示

[-0.021913960576057434, 0.006774206645786762, -0.018190348520874977, -0.039148248732089996, -0.014089343138039112]


In [17]:
'''
为刚才的文本创建embedding，准备将它们存储在向量存储中，使用向量存储上的from documents方法来实现。
该方法接受文档列表、嵌入对象，然后我们将创建一个总体向量存储
'''
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [18]:
query = "Please suggest a shirt with sunblocking"

In [19]:
docs = db.similarity_search(query)#使用这个向量存储来查找与传入查询类似的文本，如果我们在向量存储中使用相似性搜索方法并传入一个查询，我们将得到一个文档列表

In [20]:
len(docs)# 我们可以看到它返回了四个文档

4

In [21]:
docs[0] #，如果我们看第一个文档，我们可以看到它确实是一件关于防晒的衬衫

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

### 如何回答我们文档的相关问题
首先，我们需要从这个向量存储中创建一个检索器，检索器是一个通用接口，可以由任何接受查询并返回文档的方法支持。接下来，因为我们想要进行文本生成并返回自然语言响应


In [22]:
retriever = db.as_retriever() #创建检索器通用接口

In [23]:
llm = ChatOpenAI(temperature = 0.0,max_tokens=1024) #导入语言模型


In [24]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])  # 将合并文档中的所有页面内容到一个变量中


In [25]:
qdocs

': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.: 374\nname: Men\'s Plaid Tropic Shirt, Short-Sleeve\ndescription: Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offers UPF 50+ c

In [26]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one,using chinese.") #列出所有具有防晒功能的衬衫并在Markdown表格中总结每个衬衫的语言模型


In [27]:
display(Markdown(response))

| 名称 | 描述 | 
| --- | --- | 
| 太阳防护衬衫 | 高性能太阳衬衫，保证防护有害紫外线。材质为78%尼龙和22%莱卡Xtra Life纤维，UPF50+评级，手洗，晾干。吸湿快干，适合穿在泳衣上。耐磨损，可持续多季穿着。 | 
| 男士格子热带衬衫 | 轻便舒适的太阳防护衬衫，UPF50+评级，防晒效果高达98%。材质为52%聚酯纤维和48%尼龙，可机洗。前后背部有通风口，两个前口袋。 | 
| 男士夏日短袖太阳防护衬衫 | 内置UPF50+，轻盈舒适，适合炎热天气和强烈紫外线。材质为71%尼龙和29%聚酯纤维，内衬为100%聚酯纤维网眼。可机洗。防皱，前后背部有通风口，两个前口袋。 | 
| 男士热带格子短袖衬衫 | 最轻便的夏季衬衫，UPF50+评级，防晒效果高达98%。材质为100%聚酯纤维，防皱。前后背部有通风口，两个前口袋。 | 

总结：以上四款衬衫均为太阳防护衣物，UPF50+评级，防晒效果高达98%。材质轻便舒适，适合炎热天气和强烈紫外线。其中有些衬衫有防皱和通风口等特点。

在此处打印响应，我们可以看到我们得到了一个表格，正如我们所要求的那样

In [32]:
''' 
通过LangChain链封装起来
创建一个检索QA链，对检索到的文档进行问题回答，要创建这样的链，我们将传入几个不同的东西
1、语言模型，在最后进行文本生成
2、传入链类型，这里使用stuff，将所有文档塞入上下文并对语言模型进行一次调用
3、传入一个检索器
'''


qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [33]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."#创建一个查询并在此查询上运行链

In [34]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [35]:
display(Markdown(response))#使用 display 和 markdown 显示它

| Shirt Name | Description |
| --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | Rated UPF 50+ for superior protection from the sun's UV rays. Made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets. Provides the highest rated sun protection possible. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Rated to UPF 50+, helping you stay cool and dry. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. Additional features include front and back cape venting, two front bellows pockets and an imported design. With UPF 50+ coverage, you can limit sun exposure and feel secure with the highest rated sun protection available. |
| Men's TropicVibe Shirt, Short-Sleeve | Built-in UPF 50+ has the lightweight feel you want and the coverage you need when the air is hot and the UV rays are strong. Made with Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh. Wrinkle resistant. Front and back cape venting lets in cool breezes. Two front bellows pockets. Imported. |
| Sun Shield Shirt | High-performance sun shirt is guaranteed to protect from harmful UV rays. Made with 78% nylon, 22% Lycra Xtra Life fiber. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. |

All of the shirts listed have sun protection with a UPF rating of 50+ and block 98% of the sun's harmful rays. The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and has front and back cape venting and two front bellows pockets. The Men's Plaid Tropic Shirt, Short-Sleeve is made with 52% polyester and 48% nylon and has front and back cape venting and two front bellows pockets. The Men's TropicVibe Shirt, Short-Sleeve is made with Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh and has front and back cape venting and two front bellows pockets. The Sun Shield Shirt is made with 78% nylon, 22% Lycra Xtra Life fiber and fits comfortably over your favorite swimsuit.

这两个方式返回相同的结果

#### 不同类型的chain链
想在许多不同类型的块上执行相同类型的问答，该怎么办？之前的实验中只返回了4个文档，如果有多个文档，那么我们可以使用几种不同的方法
* Map Reduce   
将所有块与问题一起传递给语言模型，获取回复，使用另一个语言模型调用将所有单独的回复总结成最终答案，它可以在任意数量的文档上运行。可以并行处理单个问题，同时也需要更多的调用。它将所有文档视为独立的
* Refine    
用于循环许多文档，际上是迭代的，建立在先前文档的答案之上，非常适合前后因果信息并随时间逐步构建答案，依赖于先前调用的结果。它通常需要更长的时间，并且基本上需要与Map Reduce一样多的调用
* Map Re-rank   
对每个文档进行单个语言模型调用，要求它返回一个分数，选择最高分，这依赖于语言模型知道分数应该是什么，需要告诉它，如果它与文档相关，则应该是高分，并在那里精细调整说明，可以批量处理它们相对较快，但是更加昂贵
* Stuff    
将所有内容组合成一个文档